<a href="https://colab.research.google.com/github/eldhosejoseph/thesis/blob/main/ce901_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>